In [1]:
import folium

In [2]:
help(folium.Map)

Help on class Map in module folium.folium:

class Map(branca.element.MacroElement)
 |  Map(location=None, width='100%', height='100%', left='0%', top='0%', position='relative', tiles='OpenStreetMap', attr=None, min_zoom=0, max_zoom=18, zoom_start=10, min_lat=-90, max_lat=90, min_lon=-180, max_lon=180, max_bounds=False, crs='EPSG3857', control_scale=False, prefer_canvas=False, no_touch=False, disable_3d=False, png_enabled=False, zoom_control=True, **kwargs)
 |  
 |  Create a Map with Folium and Leaflet.js
 |  
 |  Generate a base map of given width and height with either default
 |  tilesets or a custom tileset URL. The following tilesets are built-in
 |  to Folium. Pass any of the following to the "tiles" keyword:
 |  
 |      - "OpenStreetMap"
 |      - "Mapbox Bright" (Limited levels of zoom for free tiles)
 |      - "Mapbox Control Room" (Limited levels of zoom for free tiles)
 |      - "Stamen" (Terrain, Toner, and Watercolor)
 |      - "Cloudmade" (Must pass API key)
 |      - "Ma

In [3]:
map = folium.Map(location=(-54.281327, -36.508411), zoom_start=18)

In [4]:
map.save("Map1.html")

In [5]:
map = folium.Map(tiles='http://tile.stamen.com/terrain/{z}/{x}/{y}.jpg', attr='Stamen attribution',location=(-54.281327, -36.508411), zoom_start=10)
map.save("Map2.html")

## adding layers/points

In [6]:
map = folium.Map(tiles='Stamen Terrain', location=(-54.2, -36.5))

fg = folium.FeatureGroup(name="My Map")

fg.add_child(
    folium.Marker(location=(-54.2, -36.5), popup="Start point",
                 icon=folium.Icon(
                     color='green')
                 )
)

fg.add_child(
    folium.Marker(location=(-50, -30), popup="Another point",
                 icon=folium.Icon(
                     color='green')
                 )
)

map.add_child(fg)

map.save("Map.html")

## adding points based on file and layer control

In [7]:
import numpy as np
import pandas as pd
from colour import Color

volcanoes_data = pd.read_csv(r'data\Volcanoes.txt')

In [8]:
volcanoes_data.head()

,VOLCANX020,NUMBER,NAME,LOCATION,STATUS,ELEV,TYPE,TIMEFRAME,LAT,LON
0,509.0,1201-01=,Baker,US-Washington,Historical,3285.0,Stratovolcanoes,D3,48.776798,-121.810997
1,511.0,1201-02-,Glacier Peak,US-Washington,Tephrochronology,3213.0,Stratovolcano,D4,48.111801,-121.111000
2,513.0,1201-03-,Rainier,US-Washington,Dendrochronology,4392.0,Stratovolcano,D3,46.869801,-121.751000
3,515.0,1201-05-,St. Helens,US-Washington,Historical,2549.0,Stratovolcano,D1,46.199799,-122.181000
4,516.0,1201-04-,Adams,US-Washington,Tephrochronology,3742.0,Stratovolcano,D6,46.205799,-121.490997


In [9]:
fg_volc = folium.FeatureGroup(name="Vulcanoes")
fg_pop = folium.FeatureGroup(name="Population Density")

html = """<h4>Volcano information:</h4>
Name: <a href="https://www.google.com/search?q=Volcano%%20%%22%s%%22" target="_blank">%s</a><br>
Height: %s m
"""

def get_colour_by_density(properties):
    pop = float(properties['POP2005'])
    area = float(properties['AREA']) * 10.0  #km sq
    if area == 0:
        area = 10**-10
    density = pop/area
    
    den_max = 100
    den_steps = 20
    den_dist = np.arange(0, den_max, den_steps)
    
    colours = list(Color("white").range_to(Color("red"), len(den_dist)))
    color_idx = np.argmin(np.abs(den_dist-density))
    
    return colours[color_idx].get_web()
    

for row_idx, row_data in volcanoes_data.iterrows():
    lat = row_data['LAT']
    lon = row_data['LON']
    name = row_data['NAME']
    elev = row_data['ELEV']
    
    iframe = folium.IFrame(html=html%(name, name, round(elev,1)), width=200, height=100)
    
    fg_volc.add_child(
        folium.CircleMarker(location=(lat, lon), radius=6, fill=True, fill_opacity=1,
                            fill_color='green' if elev <= 1000 else 'orange' if elev <= 3000 else 'red',
                            popup=folium.Popup(iframe), color='black', weight=1))


fg_pop.add_child(folium.GeoJson(data=open('data\world.json', encoding='utf-8-sig').read(),
                            style_function=lambda feature: {'fillColor': get_colour_by_density(feature['properties']),
                                                           'fillOpacity': 0.5}))    
map = folium.Map(tiles='Stamen Terrain', location=(lat, lon), zoom_start=2)
map.add_child(fg_volc)
map.add_child(fg_pop)

map.add_child(folium.LayerControl())

map.save("volcanoes_map.html")